## 12.1
Find a root of $\cos x$ using the three methods discussed in this section and an initial interval of $[0,10]$. Compare the solutions and the number of iterations to find the root for each method.

## Solution

The needed functions $\texttt{bisection}$, $\texttt{false\_position}$, and $\texttt{ridder}$ are defined in the file $\texttt{ch12.py}$ and imported as such. It is also acceptable to copy and paste the functions directly in the code.

$\texttt{lambda}$ function is used to define $cos~x$ as an input to the root finding methods. Each method is then executed for the range $[0,10]$ and the results printed.

In [ ]:
import numpy as np
from ch12 import *

# Define lambda function for cos(x)
cos = lambda x: np.cos(x)

# Run bisection method
print('Using bisection')
rootBi = bisection(cos,0,10)
print('Root found is',"%.7f" % rootBi)

# Run false position method
print('\nUsing false position')
rootFalse = false_position(cos,0,10)
print('Root found is',"%.7f" % rootFalse)

# Run ridder method
print('\nUsing ridder')
rootRidder = ridder(cos,0,10)
print('Root found is',"%.7f" % rootRidder)

As expected, each method found the last root of the function on $[0,10]$. The false position method preformed with a significantly fewer amount of iterations, and all roots were the same to within 6 significant figures.

## 2-D Heat Equation Optimization

Previously, we gave code to solve the heat equation:

$$-k \nabla^2 T = q, \qquad \text{for } x \in [0,L_x]\quad y\in[0,L_y].$$

With the boundary condition

$$T(x,y) = 0 \qquad \text{for } x,y \text{ on the boundary}.$$

You have been tasked to determine what value of $k$ will make the maximum temperature equal to 3 when $L_x = L_y = 1$ and the source, $q$, is given by

$$q = \begin{cases} 1 & 0.25 \leq x \leq 0.75 \qquad 0.25 \leq y \leq 0.75\\
0 & \text{otherwise}\end{cases}.$$

Your particular tasks are as follows:

- Define a function called {\tt max\_temperature} that finds the maximum value of $T(x)$ in the domain. This function will take as it's only argument $k$.  Inside the function solve the heat equation with $\Delta x = \Delta y = 0.025$. The function {\tt np.max} will be helpful here.
- Find the value of $k$ for which the  max temperature  equals 3 using bisection, false position, and Ridder's method. Use an initial interval of $k \in [0.001, 0.01]$. Remember that the root-finding methods find when a function is equal to 0. You will have to define a function that is equal to 0 when the maximum temperature is equal to 3. How many iterations does each method take?
- The Python package, {\tt time}, has a function {\tt time.clock()} that returns the system time. Using this function time how long it takes for each method to find the value of $k$ that makes the maximum temperature equal to 3. Which method is the fastest? 

This problem will demonstrate why it is important to be parsimonious with the number of function evaluations.

## Solution

First, we must use define the conjugate gradient solver.

In [ ]:
import numpy as np
import math
import time
import warnings
from ch12 import *
warnings.filterwarnings("ignore")

# CG solver from Chapter 9
def CG(A,b, x0=np.array([]),tol=1.0e-6,max_iterations=100,LOUD=False):
    """Solve a linear system by Conjugate Gradient
    Note: system must be SPD
    Args:
        A: N by N array
        b: array of length N
        x0: initial guess (if none given will be random)
        tol: Relative L2 norm tolerance for convergence
        max_iterations: maximum number of iterations
    Returns:
        The approximate solution to the linear system
    """
    
    # Create empty variable for residuals
    residuals = np.array([])
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    N = Nrow
    converged = False
    iteration = 1
    if (x0.size==0):
        x = np.random.rand(N) #random initial guess
    else:
        x = x0
    r = b - np.dot(A,x)
    s = r.copy()
    while not(converged):
        denom = np.dot(s, np.dot(A,s))
        alpha = np.dot(s,r)/denom
        x = x + alpha*s
        r = b - np.dot(A,x)
        beta = - np.dot(r,np.dot(A,s))/denom
        s = r + beta * s
        relative_change = np.linalg.norm(r)
        residuals = np.append(residuals,relative_change)
        if (LOUD):
            print("Iteration",iteration,": Relative Change =",relative_change)
        if (relative_change < tol) or (iteration >= max_iterations):
            converged = True
        iteration += 1
    return x   

A function is then made  which takes as its only input the thermal conductivity and solves the array using $\texttt{CG}$. The numpy object $\texttt{max()}$ is then used on the variable $\texttt{x}$ in order to determine the max value in the array.

In [ ]:
def max_temperature(k):
    """Solves for the temperature distribution in a simple
    square mesh and returns the maximum temperature in the
    domain
    Args:
        k: thermal conductivity
    Returns:
        The approximate maximum temperature
    """
    delta = 0.025;
    L = 1.0;
    ndim = int(round(L/delta))
    nCells = ndim*ndim;
    A = np.zeros((nCells,nCells));
    b = np.zeros(nCells)
    #save us some work for later
    idelta2 = 1.0/(delta*delta);

    #now fill in A and b
    for cellVar in range(nCells):
        xCell = cellVar % ndim; #x % y means x modulo y
        yCell = (cellVar-xCell)//ndim;
        xVal = xCell*delta + 0.5*delta;
        yVal = yCell*delta + 0.5*delta;
        #put source only in the middle of the problem
        if ( ( math.fabs(xVal - L*0.5) < .25*L) and ( math.fabs(yVal - L*0.5) < .25*L) ):
            b[cellVar] = 1;
        #end if

        A[cellVar,cellVar] = 4.0*k*idelta2;

        if (xCell > 0):
            A[cellVar,ndim*yCell + xCell -1] = -k*idelta2;
        if (xCell < ndim-1):
            A[cellVar,ndim*yCell + xCell + 1] = -k*idelta2;
        if (yCell > 0):
            A[cellVar,ndim*(yCell-1) + xCell] = -k*idelta2;
        if (yCell < ndim-1):
            A[cellVar,ndim*(yCell+1) + xCell] = -k*idelta2;

    if (nCells <= 20):
        #print the matrix
        print("The A matrix in Ax = b is\n",A)

        #print the righthand side
        print("The RHS is",b)

    x = CG(A,b,LOUD=False,max_iterations=1000)
    
    return x.max()

As root-finding methods search for the roots where the function is 0, a lambda function is defined as follows as input for the root-finding functions:

$$0 = 3.0 - T_\mathrm{max}(k).$$

The three root-finding methods defined in chapter 12 are then ran within the defined range for k, $[0.001,0.02]$. The Python $\texttt{time}$ library is used to determine the time that each set method takes.

In [ ]:
# Define function to be iterated for roots
desiredtemp = lambda k: 3.0 - max_temperature(k)

# Define bounds for root iteration
k0 = 0.001
kf = 0.02

# Run bisection method
print('Using bisection')
start = time.clock()
rootBi = bisection(desiredtemp,k0,kf)
stop = time.clock()
timeBi = stop - start
print('Root found is',"%.7f" % rootBi)
print('It took',"%.2f" % timeBi,"sec")

# Run false position method
print('\nUsing false position')
start = time.clock()
rootFalse = false_position(desiredtemp,k0,kf)
stop = time.clock()
timeFalse = stop - start
print('Root found is',"%.7f" % rootFalse)
print('It took',"%.2f" % timeFalse,"sec")

# Run ridder method
print('\nUsing Ridder')
start = time.clock()
rootRidder = ridder(desiredtemp,k0,kf)
stop = time.clock()
timeRidder = stop - start
print('Root found is',"%.7f" % rootRidder)
print('It took',"%.2f" % timeRidder,"sec")

In examination of the time study above, it is seen that the false position method performed poorly compared to the other two methods.  Of interest is that while Ridder's method only took 4 iterations, it was actually only a little faster than the bisection method. From the three methods, it was determined that the value of $k$ that results in a maximum temperature of 3 is 0.0514129.